In [7]:
import pandas as pd

In [8]:
concentration = pd.read_csv('../../data/2018/18_concentrations.csv')
labels = pd.read_csv('../../data/2018/18_yellow_labels.csv')
pit = pd.read_csv('../../data/2018/18_time_since_pit.csv')
weather = pd.read_csv('../../data/weather_data/2018_weather.csv', sep=';')

In [9]:
max_concentration = pd.DataFrame(columns=['Time', 'Max Concentration'])
temp_max = 0
for index, row in concentration.iterrows():
    sector = row[1]
    if sector == 1:
        temp_max = 0
    if row[2] > temp_max:
        temp_max = row[2]
    if sector == 13:
        max_concentration.loc[index] = [row[0], temp_max]

In [10]:
start_time = weather['TIME_UTC_SECONDS'].iloc[0]

with open('../../data/2018/18_weather_mod.csv', 'w') as f:
    f.write('Time,Air Temp,Track Temp,Humidity,Pressure,Wind Speed\n')
    for index, row in weather.iterrows():
            if row['TRACK_TEMP'] != 32:
                temp = row['TRACK_TEMP']
            else:
                row['TRACK_TEMP'] = temp
        
            f.write("%s,%s,%s,%s,%s,%s\n"%((row[0]-start_time)//20 * 20, row[2],row[3], row[4], row[5], row[6]))
            f.write("%s,%s,%s,%s,%s,%s\n"%((row[0]-start_time+20)//20 * 20,row[2],row[3], row[4], row[5], row[6]))
            f.write("%s,%s,%s,%s,%s,%s\n"%((row[0]-start_time+40)//20 * 20,row[2],row[3], row[4], row[5], row[6]))
weather_mod = pd.read_csv('../../data/2018/18_weather_mod.csv')

In [11]:
agg = max_concentration.merge(pit, how='inner', on='Time')
agg = agg.merge(weather_mod, how='inner', on='Time')
agg = agg.merge(labels, how='inner', on='Time')
agg

,Time,Max Concentration,Last Pit,Air Temp,Track Temp,Humidity,Pressure,Wind Speed,Yellow
0,20.0,27.407742,0.333333,71.90006,78.00008,62,30.269238,2.6,0
1,40.0,22.615864,0.666667,71.90006,78.00008,62,30.269238,2.6,0
2,60.0,12.329932,1.000000,71.99996,78.00008,62,30.269238,2.6,0
3,80.0,20.322355,1.333333,71.99996,78.00008,62,30.269238,2.6,0
4,100.0,11.667811,1.666667,71.99996,78.00008,62,30.269238,2.6,0
...,...,...,...,...,...,...,...,...,...
4307,86360.0,4.567957,51.820333,75.09992,78.00008,72,30.032998,1.9,0
4308,86380.0,6.280941,52.153667,75.09992,78.00008,72,30.032998,1.9,0
4309,86400.0,5.709946,52.487000,75.09992,78.00008,72,30.029159,3.2,0
4310,86420.0,6.851935,52.820333,75.09992,78.00008,72,30.029159,3.2,0


In [12]:
agg.to_csv('../../data/2018/18_agg_data.csv', index=False)